<div style="
  --ink:#eaf2ff; --muted:#a6b9d3; --panel:rgba(255,255,255,.06); --border:rgba(255,255,255,.16);
  --accent:#60a5fa; --accent2:#22d3ee; --glow:rgba(96,165,250,.40);
  background:
    radial-gradient(1200px 600px at -10% -20%, #0ea5e9 0%, #0ea5e920 40%, #0000 70%),
    radial-gradient(900px 500px at 110% 10%, #2563eb 0%, #2563eb25 45%, #0000 70%),
    linear-gradient(135deg, #0b1021 0%, #0b1428 55%, #0b1021 100%);
  color:var(--ink); border-radius:26px; padding:44px 48px; position:relative; overflow:hidden;
  box-shadow:0 22px 52px rgba(0,0,0,.45), inset 0 0 80px rgba(255,255,255,.04);
  border:1px solid var(--border); backdrop-filter:blur(14px); margin-bottom:26px;">

  <div style="position:absolute; inset:-2px; pointer-events:none;
              background: radial-gradient(1200px 1200px at 0% 100%, var(--glow), #0000 55%); opacity:.22;"></div>

  <div style="font-size:13px; letter-spacing:.3px; color:var(--muted); margin-bottom:8px;">
    Machine Learning • Mental Health • Behavioral Analytics
  </div>

  <h1 style="font-size:31px; letter-spacing:.2px; margin:0 0 10px 0;">
    🌌 <span style="color:#dbeafe;">Digital Habits & Mental Health — Predicting Wellbeing Risk</span>
  </h1>

  <p style="font-size:17px; line-height:1.75; opacity:.96; max-width:960px; margin:8px 0 10px;">
    Digital behavior leaves measurable traces of focus, fatigue, and happiness.
    This notebook turns everyday signals (screen time, unlocks, sleep balance) into
    simple, trustworthy <b>wellbeing risk</b> predictions — with a clear, beginner-friendly ML workflow.
  </p>

  <p style="font-size:16px; line-height:1.75; opacity:.94; max-width:960px; margin:0 0 10px;">
    The path is intentionally lightweight: clean data loading, concise Plotly visuals,
    baseline models you can understand, and readable evaluation you can trust.
  </p>

  <hr style="border:none;height:1px;background:linear-gradient(to right,#0000,rgba(96,165,250,.35),#0000);margin:18px 0;">

  <ul style="list-style:none;padding-left:0;margin:0;font-size:15px;line-height:1.75;opacity:.94;">
    <li>• Clean, validated loading with friendly previews.</li>
    <li>• Focused EDA: correlations & few high-value visuals (no noise).</li>
    <li>• Baseline → tuned models: Logistic Regression, Random Forest, XGBoost.</li>
    <li>• Clear metrics: AUC, Average Precision, F1, with a simple threshold pick.</li>
    <li>• Feature importance to understand what really drives risk.</li>
  </ul>

  <p style="font-size:15px;line-height:1.7;opacity:.9;margin-top:14px;font-weight:500;">
    Built for clarity and impact — minimal code, maximum insight.
  </p>

  <div style="display:flex; flex-wrap:wrap; gap:10px; margin:22px 0 0;">
    <span style="font-size:11.5px; padding:5px 11px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Simple & Clear</span>
    <span style="font-size:11.5px; padding:5px 11px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Plotly Visuals</span>
    <span style="font-size:11.5px; padding:5px 11px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Baseline → Tuned</span>
    <span style="font-size:11.5px; padding:5px 11px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Beginner-Friendly</span>
  </div>
</div>


In [1]:
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
# ===================== Minimal UI + Core Utils =====================
from IPython.display import HTML, display
import os, random, warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

# --------------------------- Config ---------------------------
CFG = dict(
    SHOW_UI=True,
    SEED=42,
    MAX_PLOT_N=1500,
    FAST_PLOTS=True,
    HEAVY_PLOTS=False,
    CV_SPLITS=5,
    TEST_SIZE=0.20,
    N_JOBS=-1,
    COST_FP=1.0,
    COST_FN=8.0,
    BENEFIT_TP=0.0,
    MAX_REVIEW_RATE=None,
    RAIN_PLOTS=True
)

# --------------------------- Theme (minimal) ---------------------------
_THEME = r"""
<style>
:root{
  --ink:#0c1224; --muted:#5e6b86; --brand:#2563eb; --accent:#06b6d4;
  --ok:#10b981; --bad:#ef4444; --panel:#ffffff; --surface:#fbfcff;
  --shadow:0 14px 28px rgba(10,15,35,.10); --border:#e8edf6;
}
@media (prefers-color-scheme: dark){
  :root{ --panel:#0f1326; --surface:#0b1021; --ink:#e6e8ef; --muted:#9aa3b2; --border:#232844; }
}
html,body{font-family:Inter,Segoe UI,system-ui,-apple-system,Arial; line-height:1.45}
.card{background:var(--panel); border:1px solid var(--border); border-radius:16px; box-shadow:var(--shadow); padding:16px; margin:10px 0}
.badge{display:inline-block; padding:3px 8px; border-radius:999px; font-size:12px; background:rgba(37,99,235,.10); color:var(--ink); border:1px solid var(--border)}
.hero{background:var(--surface); border:1px solid var(--border); box-shadow:var(--shadow); border-radius:18px; padding:16px 18px; margin:8px 0 14px}
.hero h1{margin:0; font-size:22px; color:var(--ink)}
.hero p{margin:.35rem 0 0 0; color:var(--muted); font-size:14px}
.hr{border:none; height:1px; background:linear-gradient(to right,#0000,var(--border),#0000); margin:14px 0}
table.dataframe {border:1px solid var(--border); border-radius:12px; overflow:hidden}
table.dataframe th, table.dataframe td {padding:8px 10px; border-bottom:1px solid var(--border)}
.output_png {background:var(--panel); border:1px solid var(--border); border-radius:12px; padding:6px}
</style>
"""

def _inject_theme():
    if CFG["SHOW_UI"]:
        display(HTML(_THEME))

_injected = False
if not _injected:
    _inject_theme()
    _injected = True

# --------------------------- UI ---------------------------
class UI:
    def section(self, title: str, subtitle: str | None = None):
        if CFG["SHOW_UI"]:
            sub = f"<p>{subtitle}</p>" if subtitle else ""
            display(HTML(f'<div class="hero"><h1>{title}</h1>{sub}</div><div class="hr"></div>'))
        else:
            line = "-" * len(title)
            print(f"\n{title}\n{line}")
            if subtitle: print(subtitle)

    def card_text(self, title: str | None = None, body: str = ""):
        if CFG["SHOW_UI"]:
            t = f'<span class="badge">{title}</span><br/>' if title else ""
            display(HTML(f'<div class="card">{t}{body}</div>'))
        else:
            if title: print(f"[{title}]")
            if body:  print(body)

    def card_list(self, title: str, items: list[str] | None):
        items = items or ["None"]
        if CFG["SHOW_UI"]:
            lis = "".join([f"<li>{x}</li>" for x in items])
            display(HTML(f'<div class="card"><span class="badge">{title}</span><ul>{lis}</ul></div>'))
        else:
            print(f"\n[{title}]")
            for x in items: print(f"- {x}")

    def table(self, df: pd.DataFrame, title: str | None = None, head: int = 5):
        if CFG["SHOW_UI"]:
            t = f'<span class="badge">{title}</span>' if title else ""
            display(HTML(f'<div class="card">{t}{df.head(head).to_html(index=False)}</div>'))
        else:
            if title: print(f"[{title}]")
            print(df.head(head).to_string(index=False))

    def hr(self):
        if CFG["SHOW_UI"]:
            display(HTML('<div class="hr"></div>'))
        else:
            print("-"*60)

ui = UI()

# --------------------------- Helpers ---------------------------
def section(title: str, subtitle: str | None = None): ui.section(title, subtitle)
def show_list(title: str, items: list[str] | None): ui.card_list(title, items)
def show_table(title: str, df: pd.DataFrame, head: int = 10): ui.table(df, title=title, head=head)

def set_global_seeds(seed: int = 42):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    try:
        import torch
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except Exception:
        pass

set_global_seeds(CFG["SEED"])

def thresholds_report(y_true, proba, steps: int = 401) -> pd.DataFrame:
    y_true = np.asarray(y_true, dtype=int)
    proba  = np.asarray(proba, dtype=float)
    ts = np.linspace(0.0, 1.0, steps)
    rows = []
    for t in ts:
        pred = (proba >= t).astype(int)
        tp = int(((pred==1)&(y_true==1)).sum())
        fp = int(((pred==1)&(y_true==0)).sum())
        fn = int(((pred==0)&(y_true==1)).sum())
        precision = tp/(tp+fp+1e-9); recall = tp/(tp+fn+1e-9)
        f1 = 2*precision*recall/(precision+recall+1e-9)
        rows.append((float(t), precision, recall, f1))
    return pd.DataFrame(rows, columns=["threshold","precision","recall","f1"])

# XGBoost device-probe
def xgb_device_params(verbose: bool = False):
    import xgboost as xgb
    try:
        ver = tuple(int(x) for x in xgb.__version__.split(".")[:2])
    except Exception:
        ver = (2, 0)
    cuda_params = {"tree_method": "hist", "device": "cuda"} if ver >= (2, 0) else {"tree_method": "gpu_hist"}
    cpu_params  = {"tree_method": "hist", "device": "cpu"}  if ver >= (2, 0) else {"tree_method": "hist"}
    try:
        d = xgb.DMatrix(np.array([[0.0]], dtype=np.float32), label=np.array([0], dtype=np.float32))
        xgb.train(params={**cuda_params, "max_depth": 1, "learning_rate": 0.1, "nthread": 1, "verbosity": 0},
                  dtrain=d, num_boost_round=1)
        if verbose: ui.card_text("XGBoost Device", f"Using GPU (xgboost {xgb.__version__})")
        return cuda_params
    except Exception as e:
        if verbose: ui.card_text("XGBoost Device", f"Falling back to CPU (xgboost {xgb.__version__}) — {str(e).splitlines()[0]}")
        return cpu_params

section("Notebook Setup", "UI loaded")

In [3]:
# ===================== Data Loading (robust, single read, validated) =====================
assert all(k in globals() for k in ["ui","section","show_list","show_table","CFG"]), "Run the setup block first."

# --------------------------- Config ---------------------------
if "TARGET" not in globals():
    TARGET = "high_risk_flag"

TARGET_MAP = None

CANDIDATE_PATHS = [
   "/kaggle/input/digital-health-and-mental-wellness/Data.csv",
]

# --------------------------- Helpers ---------------------------
def find_data_path(candidates: list[str], env_var: str | None = None) -> str | None:
    if env_var:
        env_path = os.environ.get(env_var, "").strip()
        if env_path and os.path.exists(env_path):
            return env_path
    for p in candidates:
        if p and os.path.exists(p):
            return p
    return None

def _sanitize_columns(d: pd.DataFrame) -> pd.DataFrame:
    d = d.copy()
    d.columns = (
        d.columns.astype(str)
        .str.strip()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^0-9A-Za-z_]", "", regex=True)
    )
    return d

def schema_overview(d: pd.DataFrame, n_unique_cap: int = 999999) -> pd.DataFrame:
    out = []
    for c in d.columns:
        s = d[c]
        miss = float(s.isna().mean()) * 100.0
        nun  = int(min(s.nunique(dropna=True), n_unique_cap))
        try:
            ex = ", ".join(map(lambda x: str(x)[:24], s.dropna().astype(str).unique()[:3]))
        except Exception:
            ex = ""
        out.append({"column": c, "dtype": str(s.dtype), "missing_%": round(miss, 2), "n_unique": nun, "examples": ex})
    return pd.DataFrame(out).sort_values(["missing_%", "column"], ascending=[False, True])

# --------------------------- Locate & read ---------------------------
DATA_PATH = find_data_path(CANDIDATE_PATHS, env_var="DWELL_DATA")
if DATA_PATH is None:
    raise FileNotFoundError("❌ Dataset not found. Set DWELL_DATA or update CANDIDATE_PATHS.")

df = pd.read_csv(DATA_PATH, low_memory=False)
df = _sanitize_columns(df)

dup_cnt = int(df.duplicated().sum())
if dup_cnt:
    df = df.drop_duplicates().reset_index(drop=True)

# --------------------------- Target validation ---------------------------
if TARGET not in df.columns:
    raise ValueError(f"❌ TARGET '{TARGET}' not found in dataframe columns.")

if TARGET_MAP is not None:
    df[TARGET] = df[TARGET].map(TARGET_MAP)
    if df[TARGET].isna().any():
        bad_examples = df.loc[df[TARGET].isna()].iloc[:5, :].to_dict(orient="records")
        raise ValueError(f"❌ TARGET_MAP left unmapped values. Examples: {bad_examples}")
    df[TARGET] = df[TARGET].astype(int)
else:
    if pd.api.types.is_bool_dtype(df[TARGET]):
        df[TARGET] = df[TARGET].astype(int)
    elif pd.api.types.is_numeric_dtype(df[TARGET]):
        unique_vals = set(pd.Series(df[TARGET].dropna().unique()).tolist())
        if not unique_vals.issubset({0, 1}):
            raise ValueError(
                f"❌ TARGET '{TARGET}' must be binary 0/1 or provide TARGET_MAP. Found: {sorted(unique_vals)[:10]}"
            )
        df[TARGET] = df[TARGET].astype(int)
    else:
        raise ValueError(
            f"❌ Non-numeric TARGET detected. Set TARGET_MAP to define binary mapping, e.g. TARGET_MAP = {{'Low':0,'High':1}}."
        )

# --------------------------- Snapshot & guards ---------------------------
os.makedirs("artifacts/eda", exist_ok=True)
schema_df = schema_overview(df)
schema_df.to_csv("artifacts/eda/schema_overview.csv", index=False)

nun = df.nunique(dropna=True)
near_id_like = [c for c in df.columns if nun[c] > 0.95 * len(df)]
constant_cols = [c for c in df.columns if nun[c] <= 1]
prevalence = float(df[TARGET].mean()) * 100.0

# --------------------------- Report ---------------------------
section("Data Loaded", "📦 Path • Shape • Target • Duplicates • Schema snapshot")
show_list("Load Report", [
    f"Path: {DATA_PATH}",
    f"Rows × Cols: {df.shape[0]:,} × {df.shape[1]:,}",
    f"Duplicates dropped: {dup_cnt:,}",
    f"Target: {TARGET}",
    f"Positive prevalence: {prevalence:.2f}%",
    f"Schema saved → artifacts/eda/schema_overview.csv",
])
if near_id_like: show_list("Potential ID-like columns", near_id_like)
if constant_cols: show_list("Constant columns", constant_cols)
show_table("Head (preview)", df, head=5)

# --------------------------- Expose for downstream blocks ---------------------------
NUM_COLS_ALL = df.select_dtypes(include=[np.number]).columns.tolist()
CAT_COLS_ALL = [c for c in df.columns if c not in NUM_COLS_ALL]
FEATURES = [c for c in df.columns if c != TARGET]


id,age,gender,region,income_level,education_level,daily_role,device_hours_per_day,phone_unlocks,notifications_per_day,social_media_mins,study_mins,physical_activity_days,sleep_hours,sleep_quality,anxiety_score,depression_score,stress_level,happiness_score,focus_score,high_risk_flag,device_type,productivity_score,digital_dependence_score
1,40,Female,Asia,High,High School,Part-time/Shift,3.54,45,561,98,34,7.0,9.123800,3.353627,9.926651,5.0,6.593289,8.0,23.0,0,Android,70.000000,25.700000
2,27,Male,Africa,Lower-Mid,Master,Full-time Employee,5.65,100,393,174,102,2.0,8.837517,2.908147,4.000000,4.0,4.126926,8.1,35.0,0,Laptop,64.000000,30.100000
3,31,Male,North America,Lower-Mid,Bachelor,Full-time Employee,8.87,181,231,595,140,1.0,6.486743,2.889213,4.000000,8.0,1.429139,7.6,15.0,0,Android,65.299301,40.600000
4,41,Female,Middle East,Low,Master,Caregiver/Home,4.05,94,268,18,121,4.0,7.600504,3.097488,7.093357,9.0,4.995512,7.8,28.0,1,Tablet,80.000000,36.684152
5,26,Female,Europe,Lower-Mid,Bachelor,Full-time Employee,13.07,199,91,147,60,1.0,5.197962,2.786098,7.028125,15.0,9.448757,4.2,70.0,1,Android,65.299301,48.400000


In [4]:
# ===================== EDA (Plotly) — Leakage • Correlation • Distributions =====================
import plotly.express as px, plotly.graph_objects as go
from scipy.stats import spearmanr
import numpy as np, pandas as pd

assert all(k in globals() for k in ["df","TARGET","section","show_list","show_table","CFG","NUM_COLS_ALL","CAT_COLS_ALL"]), \
    "❌ Run setup and data-loading blocks first."

SEED       = CFG.get("SEED", 42)
FAST_PLOTS = CFG.get("FAST_PLOTS", True)
MAX_PLOT_N = CFG.get("MAX_PLOT_N", 1500)
RAIN_PLOTS = CFG.get("RAIN_PLOTS", True)

rng = np.random.default_rng(SEED)

def maybe_sample(d: pd.DataFrame, n: int) -> pd.DataFrame:
    if FAST_PLOTS and len(d) > n:
        idx = rng.choice(len(d), size=n, replace=False)
        return d.iloc[idx].copy()
    return d

SAMPLE = maybe_sample(df, MAX_PLOT_N)
feat_num = [c for c in NUM_COLS_ALL if c != TARGET]

section("📊 Exploratory Data Analysis", "🔎 Leakage checks • Correlations • Distributions")

# --------------------------- Leakage Checks ---------------------------
def leakage_guard(d: pd.DataFrame, target_col: str, top_k: int = 12):
    issues = {}
    nun = d.nunique(dropna=True)
    near_id = [c for c in d.columns if nun[c] > 0.95 * len(d)]
    if near_id: issues["near_id_like"] = near_id
    constant = [c for c in d.columns if nun[c] <= 1]
    if constant: issues["constant"] = constant
    if pd.api.types.is_numeric_dtype(d[target_col]):
        corrs = {}
        for c in d.columns:
            if c == target_col or not pd.api.types.is_numeric_dtype(d[c]):
                continue
            try:
                r, _ = spearmanr(d[c], d[target_col], nan_policy="omit")
                if np.isfinite(r): corrs[c] = abs(float(r))
            except: pass
        top = sorted(corrs.items(), key=lambda x: -x[1])[:top_k]
        issues["top_target_corr"] = top
    return issues

leak_report = leakage_guard(df, TARGET, top_k=15)
leak_items = []
if leak_report.get("near_id_like"): leak_items.append("ID-like: " + ", ".join(leak_report["near_id_like"]))
if leak_report.get("constant"):     leak_items.append("Constant: " + ", ".join(leak_report["constant"]))
if leak_report.get("top_target_corr"):
    leak_items += [f"{c}: {v:.3f}" for c, v in leak_report["top_target_corr"]]
show_list("🧪 Leakage Guard", leak_items or ["No obvious issues found."])

# --------------------------- Correlation Heatmap ---------------------------
TOP_NUMERIC = 10
if pd.api.types.is_numeric_dtype(SAMPLE[TARGET]) and len(feat_num) > 0:
    corr_to_target = (
        SAMPLE[feat_num + [TARGET]]
        .corr(method="spearman", numeric_only=True)[TARGET]
        .drop(TARGET).abs().sort_values(ascending=False)
    )
    top_feats = corr_to_target.index.tolist()[:TOP_NUMERIC]
else:
    top_feats = feat_num[:TOP_NUMERIC]

heat_cols = [TARGET] + [c for c in top_feats if c != TARGET]
if len(heat_cols) >= 2:
    corr = SAMPLE[heat_cols].corr(method="spearman", numeric_only=True)
    order = corr[TARGET].abs().sort_values(ascending=False).index.tolist()
    corr = corr.loc[order, order]
    fig = px.imshow(
        corr, text_auto=".2f", aspect="auto",
        color_continuous_scale="RdBu_r", zmin=-1, zmax=1,
        title="Spearman Correlation Heatmap"
    )
    fig.update_layout(height=500, width=700, margin=dict(l=50, r=40, t=60, b=40))
    fig.show()

# --------------------------- Raincloud Plots ---------------------------
def raincloud_plot(d: pd.DataFrame, target_col: str, value_col: str, title: str):
    if target_col not in d.columns or value_col not in d.columns: return
    fig = go.Figure()
    for cls, name, color in [(0, "Low Risk", "#1f77b4"), (1, "High Risk", "#d62728")]:
        sub = d[d[target_col] == cls]
        if len(sub) == 0: continue
        fig.add_trace(go.Violin(
            y=sub[value_col], name=name, box_visible=True, meanline_visible=True,
            points="all" if not FAST_PLOTS else False, jitter=0.25,
            scalemode="count", line_color=color, fillcolor=color, opacity=0.4
        ))
    fig.update_layout(title=title, violingap=0.3, height=380, width=700)
    fig.show()

if RAIN_PLOTS and leak_report.get("top_target_corr"):
    top_num_for_rain = [c for c, _ in leak_report["top_target_corr"] if c in feat_num][:4]
    for c in top_num_for_rain:
        try:
            raincloud_plot(SAMPLE, TARGET, c, f"{c} vs {TARGET} • Distribution by Class")
        except: pass

# --------------------------- Numeric Histograms ---------------------------
top_for_plot = [c for c, _ in leak_report.get("top_target_corr", []) if c in feat_num][:6]
for c in top_for_plot:
    try:
        fig = px.histogram(
            SAMPLE, x=c, color=TARGET, nbins=50, barmode="overlay", opacity=0.6,
            title=f"{c} vs {TARGET} • Histogram"
        )
        fig.update_layout(height=380, width=700, margin=dict(l=40, r=40, t=50, b=40))
        fig.update_traces(marker_line_width=0)
        fig.show()
    except: pass

# --------------------------- Categorical Plots ---------------------------
MAX_CAT_PLOT = min(6, len(CAT_COLS_ALL))
cat_for_plot = [c for c in CAT_COLS_ALL if df[c].nunique(dropna=True) > 1][:MAX_CAT_PLOT]

for c in cat_for_plot:
    try:
        tmp = SAMPLE[[c, TARGET]].dropna()
        if tmp.empty: continue
        top_lvls = tmp[c].value_counts().index[:12]
        tmp = tmp[tmp[c].isin(top_lvls)].copy()
        rate = tmp.groupby(c, observed=True)[TARGET].mean().reset_index().rename(columns={TARGET: "rate"})
        cnt  = tmp[c].value_counts().reset_index().rename(columns={"index": c, c: "count"})

        fig1 = px.bar(rate, x=c, y="rate", title=f"Rate by {c}")
        fig1.update_layout(yaxis_tickformat=".1%", height=350, width=650)
        fig1.show()

        fig2 = px.bar(cnt, x=c, y="count", title=f"Count by {c}")
        fig2.update_layout(height=350, width=650)
        fig2.show()
    except: pass

# --------------------------- Parallel Categories ---------------------------
cats = [c for c in CAT_COLS_ALL if df[c].nunique(dropna=True) <= 10][:3]
if len(cats) >= 2:
    tmp = SAMPLE[cats + [TARGET]].dropna().copy()
    tmp[TARGET] = tmp[TARGET].astype(int)
    if len(tmp) > 0:
        fig = px.parallel_categories(
            tmp, dimensions=cats, color=TARGET,
            color_continuous_scale="Reds", title="Cohort Slicer"
        )
        fig.update_layout(height=450, width=800)
        fig.show()

# --------------------------- Sunburst ---------------------------
cat_small = [c for c in CAT_COLS_ALL if df[c].nunique(dropna=True) <= 8]
if len(cat_small) >= 2:
    dims = cat_small[:2]
    g = SAMPLE.groupby(dims, observed=True)[TARGET].mean().reset_index().rename(columns={TARGET: "risk_rate"})
    if not g.empty:
        fig = px.sunburst(
            g, path=dims, values="risk_rate", color="risk_rate",
            color_continuous_scale="Reds", title="Hierarchical Risk Rate"
        )
        fig.update_layout(height=450, width=700)
        fig.show()

# --------------------------- Pairwise Scatter ---------------------------
def top_corr_features(d: pd.DataFrame, target_col: str, k: int = 6):
    scores = []
    for c in d.columns:
        if c == target_col or not pd.api.types.is_numeric_dtype(d[c]): continue
        try:
            r, _ = spearmanr(d[c], d[target_col], nan_policy="omit")
            if np.isfinite(r): scores.append((c, abs(float(r))))
        except: pass
    scores.sort(key=lambda x: -x[1])
    return [c for c, _ in scores[:k]]

if len(feat_num) >= 2:
    topk = top_corr_features(SAMPLE[feat_num + [TARGET]], TARGET, k=min(5, len(feat_num)))
    pairs = [(topk[i], topk[j]) for i in range(len(topk)) for j in range(i+1, len(topk))][:4]
    for a, b in pairs:
        try:
            fig = px.scatter(SAMPLE, x=a, y=b, color=TARGET, opacity=0.55, title=f"{a} vs {b}")
            fig.update_layout(height=400, width=700)
            fig.show()
        except: pass

# --------------------------- Summary ---------------------------
notes = []
notes.append(f"Prevalence: {float(df[TARGET].mean())*100:.2f}% positive class.")
if leak_report.get("top_target_corr"):
    top_corr = ", ".join([f"{c} ({v:.2f})" for c, v in leak_report["top_target_corr"][:5]])
    notes.append(f"Top numeric signals: {top_corr}.")
if cat_for_plot: notes.append(f"Categorical inspected: {len(cat_for_plot)}")
notes.append(f"Sampling: {'on' if FAST_PLOTS else 'off'} (N={len(SAMPLE)})")
notes.append(f"Raincloud: {'on' if RAIN_PLOTS else 'off'}")
show_list("📌 EDA Summary", notes)


In [5]:
# ===================== Feature Engineering — interactions • ratios • normalization =====================
from scipy.stats import spearmanr
import numpy as np, pandas as pd
import os

assert "df" in globals() and "TARGET" in globals(), "❌ Run previous blocks first."

def forbid_target_leak_by_name(columns, target):
    leaked = [c for c in columns if c != target and target.lower() in c.lower()]
    if leaked:
        raise ValueError(f"Potential leakage: feature name(s) include target: {leaked}")

forbid_target_leak_by_name(df.columns, TARGET)

def winsorize(s: pd.Series, lo: float = 0.01, hi: float = 0.99) -> pd.Series:
    if not pd.api.types.is_numeric_dtype(s): return s
    lo_val, hi_val = s.quantile(lo), s.quantile(hi)
    return s.clip(lo_val, hi_val)

def add_engineered_features(d: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    d = d.copy()
    new_cols = {}

    # --- Interactions & Ratios ---
    if {"device_hours_per_day", "sleep_hours"}.issubset(d.columns):
        new_cols["screen_per_sleep_norm"] = (d["device_hours_per_day"] / d["sleep_hours"].replace(0, np.nan)).fillna(0).clip(0, 8)

    if {"stress_level", "sleep_hours"}.issubset(d.columns):
        new_cols["stress_sleep_interaction"] = d["stress_level"] * (10 - d["sleep_hours"]).clip(lower=0)

    if {"anxiety_score", "device_hours_per_day"}.issubset(d.columns):
        new_cols["anxiety_screen_interaction"] = d["anxiety_score"] * d["device_hours_per_day"]

    if {"focus_score", "stress_level"}.issubset(d.columns):
        new_cols["productivity_gap"] = d["focus_score"] - d["stress_level"]

    if {"physical_activity_days", "happiness_score"}.issubset(d.columns):
        new_cols["active_wellbeing"] = d["physical_activity_days"] * d["happiness_score"]

    if {"phone_unlocks", "device_hours_per_day"}.issubset(d.columns):
        new_cols["unlocks_per_hour"] = (d["phone_unlocks"] / (d["device_hours_per_day"] + 0.1)).clip(0, 120)

    if {"social_media_minutes", "work_minutes"}.issubset(d.columns):
        total_m = (d["social_media_minutes"].fillna(0) + d["work_minutes"].fillna(0)).replace(0, np.nan)
        new_cols["social_ratio"] = (d["social_media_minutes"] / total_m).fillna(0).clip(0, 1)
        new_cols["work_ratio"]   = (d["work_minutes"] / total_m).fillna(0).clip(0, 1)

    # --- Winsorized z-scores ---
    z_cols = [c for c in ["device_hours_per_day", "sleep_hours", "stress_level",
                          "anxiety_score", "depression_score", "happiness_score"] if c in d.columns]
    for c in z_cols:
        s = winsorize(d[c].astype("float32"))
        new_cols[f"z_{c}"] = (s - s.mean()) / (s.std(ddof=0) + 1e-9)

    # --- Heavy-tailed log transforms ---
    for c in ["phone_unlocks", "social_media_minutes", "work_minutes", "notifications_per_day"]:
        if c in d.columns and pd.api.types.is_numeric_dtype(d[c]):
            new_cols[f"log1p_{c}"] = np.log1p(d[c].clip(lower=0))

    # Attach to dataframe
    for k, v in new_cols.items():
        d[k] = pd.Series(v, index=d.index)
        if pd.api.types.is_numeric_dtype(d[k]):
            d[k] = d[k].astype("float32")

    return d, list(new_cols.keys())

df, ENG_FEATURES = add_engineered_features(df)

# --------------------------- Quality & Correlation Report ---------------------------
def engineered_quality_report(d: pd.DataFrame, eng_cols: list[str], target: str):
    rep = []
    for c in eng_cols:
        s = d[c]
        rep.append({
            "feature": c,
            "missing_%": round(100 * float(s.isna().mean()), 2),
            "n_unique": int(s.nunique(dropna=True))
        })
    rep = pd.DataFrame(rep).sort_values(["missing_%", "feature"])

    if pd.api.types.is_numeric_dtype(d[target]):
        corrs = []
        for c in eng_cols:
            try:
                r, _ = spearmanr(d[c], d[target], nan_policy="omit")
                corrs.append((c, float(r)))
            except: corrs.append((c, np.nan))
        corrs = pd.DataFrame(corrs, columns=["feature", "corr_vs_target"]).sort_values(
            "corr_vs_target", key=lambda s: s.abs(), ascending=False)
    else:
        corrs = pd.DataFrame({"feature": eng_cols, "corr_vs_target": np.nan})

    return rep, corrs

eng_quality, eng_corr = engineered_quality_report(df, ENG_FEATURES, TARGET)

section("🧠 Feature Engineering", "✔ Interactions • Ratios • Winsorized z-scores • Log transforms")
show_list("Summary", [f"Added features: {len(ENG_FEATURES)}"] if ENG_FEATURES else ["No engineered features added."])

# Save artifacts
os.makedirs("artifacts/eda", exist_ok=True)
eng_quality.to_csv("artifacts/eda/engineered_quality.csv", index=False)
eng_corr.to_csv("artifacts/eda/engineered_corr.csv", index=False)

# --------------------------- Visualize Top Signals ---------------------------
try:
    topk = eng_corr.dropna(subset=["corr_vs_target"]).head(15)
    if len(topk):
        import plotly.express as px
        fig = px.bar(
            topk.sort_values("corr_vs_target", key=lambda s: s.abs()),
            x="corr_vs_target", y="feature", orientation="h",
            title="Top Engineered Features — Correlation vs Target"
        )
        fig.update_layout(height=500, width=700, margin=dict(l=50, r=40, t=60, b=40))
        fig.show()
except: pass

show_table("🧪 Engineered Features — Quality Snapshot", eng_quality, head=20)

# --------------------------- Modeling Columns ---------------------------
DROP_CANDIDATES = ["id", "screen_to_sleep_ratio", "social_vs_productivity_ratio", "wellbeing_index"]
drop_list = [c for c in DROP_CANDIDATES if c in df.columns]

nun = df.nunique(dropna=True)
constant_cols = [c for c in df.columns if nun[c] <= 1 and c != TARGET]
drop_list.extend([c for c in constant_cols if c not in drop_list])

FEATURES = [c for c in df.columns if c not in drop_list + [TARGET]]
num_cols = [c for c in FEATURES if pd.api.types.is_numeric_dtype(df[c])]
cat_cols = [c for c in FEATURES if c not in num_cols]

show_list("🧮 Columns", [
    f"Using: {len(FEATURES)} features",
    f"Numeric: {len(num_cols)}",
    f"Categorical: {len(cat_cols)}",
    f"Dropped constants: {len(constant_cols)}"
])


feature,missing_%,n_unique
active_wellbeing,0.0,372
anxiety_screen_interaction,0.0,3106
log1p_notifications_per_day,0.0,802
log1p_phone_unlocks,0.0,324
productivity_gap,0.0,2570
screen_per_sleep_norm,0.0,3500
stress_sleep_interaction,0.0,3458
unlocks_per_hour,0.0,3260
z_anxiety_score,0.0,1759
z_depression_score,0.0,24


In [6]:
# ===================== A) Modeling & Evaluation — CV-tuned, Calibrated, Visuals =====================
import os, json, time, joblib, numpy as np, pandas as pd
import plotly.express as px, plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss,
    roc_curve, precision_recall_curve
)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV
from scipy.stats import chi2_contingency

# ---- Guards & UI ----
assert all(k in globals() for k in ["df","TARGET","FEATURES","num_cols","cat_cols","section","show_list","xgb_device_params","CFG"]), \
    "Run setup & feature-engineering blocks first."

try:
    ui
except NameError:
    class _UI:
        def card_text(self, title, body=""): print(f"[{title}] {body}")
    ui = _UI()

# ---- Config ----
SEED       = CFG.get("SEED", 42)
CV_SPLITS  = CFG.get("CV_SPLITS", 5)
TEST_SIZE  = CFG.get("TEST_SIZE", 0.20)
N_JOBS     = CFG.get("N_JOBS", -1)

os.makedirs("artifacts/metrics", exist_ok=True)
os.makedirs("artifacts/decision", exist_ok=True)

# ---- Device params for XGBoost ----
if "DEVICE_PARAMS" not in globals():
    DEVICE_PARAMS = xgb_device_params(verbose=False)
ui.card_text("XGBoost Device", f"Using params: <code>{DEVICE_PARAMS}</code>")

section("🚦 Modeling & Evaluation", "CV selection • Calibration • Leaderboard • ROC/PR • Reliability • Lift")

# ---- Split & rare-binning ----
X = df[FEATURES].copy()
y = df[TARGET].astype(int)
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=TEST_SIZE, stratify=y, random_state=SEED)
cv = StratifiedKFold(n_splits=CV_SPLITS, shuffle=True, random_state=SEED)
pos_ratio = float(y_tr.mean())
scale_pos = float((1 - pos_ratio) / max(pos_ratio, 1e-9))
show_list("Class Balance", [f"Train positive rate: {pos_ratio*100:.2f}%", f"XGB scale_pos_weight ≈ {scale_pos:.2f}"])

def learn_rare_bins(d, cat_cols, min_frac=0.01):
    maps = {}
    n = len(d)
    for c in cat_cols:
        if c not in d.columns:
            continue
        vc = d[c].value_counts(dropna=False) / max(n, 1)
        rare = set(vc[vc < min_frac].index.tolist())
        if len(rare):
            maps[c] = rare
    return maps

def apply_rare_bins(d, maps, other_token="__OTHER__"):
    d = d.copy()
    for c, rare in maps.items():
        if c in d.columns:
            d[c] = d[c].astype("object").where(~d[c].isin(rare), other_token)
    return d

cat_cols_pre = [c for c in cat_cols if c in X_tr.columns]
rare_maps = learn_rare_bins(X_tr, cat_cols_pre)
X_tr, X_te = apply_rare_bins(X_tr, rare_maps), apply_rare_bins(X_te, rare_maps)
with open("artifacts/metrics/rare_maps.json", "w") as f:
    json.dump({k: list(v) for k, v in rare_maps.items()}, f, indent=2)
ui.card_text("Rare-binning", "Saved mapping → artifacts/metrics/rare_maps.json")

# Target stability (sanity)
tbl = pd.crosstab(index=pd.Series(["train"]*len(y_tr) + ["test"]*len(y_te), name="split"),
                  columns=pd.Series(pd.concat([y_tr, y_te]).values, name="target"))
_, p_stab, _, _ = chi2_contingency(tbl)
show_list("Target Stability", [f"Train+Test chi² p-value: {p_stab:.3g}"])

# ---- Preprocessing ----
num_cols_fit = [c for c in X_tr.columns if pd.api.types.is_numeric_dtype(X_tr[c])]
cat_cols_fit = [c for c in X_tr.columns if c not in num_cols_fit]
numeric_tf = Pipeline([("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler(with_mean=False))])
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)
categorical_tf = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")), ("ohe", ohe)])
pre = ColumnTransformer([("num", numeric_tf, num_cols_fit), ("cat", categorical_tf, cat_cols_fit)], remainder="drop")

# ---- Models ----
logreg = Pipeline([("pre", pre), ("clf", LogisticRegression(max_iter=2000, solver="saga",
                    class_weight="balanced", random_state=SEED, n_jobs=N_JOBS))])
rf = Pipeline([("pre", pre), ("clf", RandomForestClassifier(n_estimators=900, class_weight="balanced_subsample",
                    random_state=SEED, n_jobs=N_JOBS))])
xgb = Pipeline([("pre", pre), ("clf", XGBClassifier(n_estimators=1200, learning_rate=0.045, max_depth=6,
                    subsample=0.9, colsample_bytree=0.9, reg_lambda=1.0, objective="binary:logistic",
                    eval_metric="auc", random_state=SEED, n_jobs=N_JOBS, scale_pos_weight=scale_pos, **DEVICE_PARAMS))])
MODELS = {"LogisticRegression": logreg, "RandomForest": rf, "XGBoost": xgb}

# ---- CV select + test eval (uncalibrated) ----
def eval_model(name, model):
    t0 = time.time()
    auc_cv = float(cross_val_score(model, X_tr, y_tr, scoring="roc_auc", cv=cv, n_jobs=N_JOBS).mean())
    model.fit(X_tr, y_tr)
    proba = model.predict_proba(X_te)[:, 1]
    return name, {"auc_cv": auc_cv, "auc_te": roc_auc_score(y_te, proba),
                  "ap_te": average_precision_score(y_te, proba),
                  "brier": brier_score_loss(y_te, proba),
                  "proba": proba, "model": model, "train_s": round(time.time() - t0, 2)}

SCORES = dict(eval_model(n, m) for n, m in MODELS.items())
best_name = max(SCORES, key=lambda k: SCORES[k]["auc_cv"])
best_uncal = SCORES[best_name]["model"]
ui.card_text("Selected Best (CV)", f"<b>{best_name}</b> with AUC_CV={SCORES[best_name]['auc_cv']:.3f}")

# ---- Calibration on the best ----
cal = CalibratedClassifierCV(best_uncal, method="isotonic", cv=3)
cal.fit(X_tr, y_tr)
best_model = cal
proba_best = best_model.predict_proba(X_te)[:, 1]
cal_auc, cal_ap = roc_auc_score(y_te, proba_best), average_precision_score(y_te, proba_best)
joblib.dump(best_model, "artifacts/decision/best_model_calibrated.joblib")
ui.card_text("Model Saved", "artifacts/decision/best_model_calibrated.joblib")

# ---- Leaderboard (replace best test scores with calibrated) ----
lb = pd.DataFrame([
    {"model": n, "AUC_CV": v["auc_cv"], "AUC_Test": v["auc_te"], "AP_Test": v["ap_te"],
     "Brier_Test": v["brier"], "Gini_Test": 2*v["auc_te"] - 1, "Train_s": v["train_s"]}
    for n, v in SCORES.items()
]).sort_values("AUC_CV", ascending=False).reset_index(drop=True)
lb.loc[lb["model"] == best_name, ["AUC_Test","AP_Test"]] = [cal_auc, cal_ap]
lb.to_csv("artifacts/metrics/leaderboard_metrics.csv", index=False)

fig = go.Figure()
fig.add_bar(x=lb["model"], y=lb["AUC_CV"], name="AUC (CV)")
fig.add_bar(x=lb["model"], y=lb["AUC_Test"], name="AUC (Test)")
fig.add_bar(x=lb["model"], y=lb["AP_Test"], name="AP (Test)")
fig.update_layout(barmode="group", title="Model Leaderboard — CV vs Test", yaxis_title="Score")
fig.show()

# ---- ROC & PR (all models, uncalibrated) ----
FIG_W, FIG_H = 1200, 520
COLORS = (px.colors.qualitative.Set2 + px.colors.qualitative.Set1 + px.colors.qualitative.Pastel)
def color_of(i): return COLORS[i % len(COLORS)]

fig = make_subplots(rows=1, cols=2, subplot_titles=("ROC — All Models (uncalibrated)", "Precision–Recall — All Models (uncalibrated)"))
for i, (name, v) in enumerate(SCORES.items()):
    p = v["proba"]
    fpr, tpr, _ = roc_curve(y_te, p)
    prc, rc, _  = precision_recall_curve(y_te, p)
    roc_auc, ap = roc_auc_score(y_te, p), average_precision_score(y_te, p)
    fig.add_scatter(x=fpr, y=tpr, mode="lines", name=f"{name} (AUC={roc_auc:.3f})",
                    line=dict(color=color_of(i), width=2.0), row=1, col=1)
    fig.add_scatter(x=rc, y=prc, mode="lines", name=f"{name} (AP={ap:.3f})",
                    line=dict(color=color_of(i), width=2.0), row=1, col=2, showlegend=False)
fig.add_scatter(x=[0,1], y=[0,1], mode="lines", name="Chance", line=dict(dash="dash", color="#888", width=1.2), row=1, col=1)
fig.update_xaxes(title_text="FPR", row=1, col=1); fig.update_yaxes(title_text="TPR", row=1, col=1)
fig.update_xaxes(title_text="Recall", row=1, col=2); fig.update_yaxes(title_text="Precision", row=1, col=2)
fig.update_layout(title="Models — Comparative ROC & PR", width=FIG_W, height=FIG_H,
                  legend=dict(orientation="v", yanchor="top", y=0.98, xanchor="left", x=1.02, bgcolor="rgba(0,0,0,0)"),
                  margin=dict(l=60, r=180, t=60, b=50))
fig.show()

# ---- Reliability curve (calibrated best) & Lift curve ----
def reliability_curve(y_true, proba, bins=10):
    y_true, proba = np.asarray(y_true, int), np.asarray(proba, float)
    edges = np.linspace(0.0, 1.0, bins + 1)
    inds = np.clip(np.digitize(proba, edges, right=True) - 1, 0, bins - 1)
    rows = []
    for b in range(bins):
        mask = inds == b
        if mask.any():
            rows.append((proba[mask].mean(), y_true[mask].mean(), int(mask.sum())))
    return pd.DataFrame(rows, columns=["mean_pred", "frac_pos", "count"]).sort_values("mean_pred")

def lift_gain_curves(y_true, proba, bins=20):
    dfp = pd.DataFrame({"y": np.asarray(y_true, int), "p": proba}).sort_values("p", ascending=False).reset_index(drop=True)
    dfp["bucket"] = pd.qcut(dfp.index, q=bins, labels=False, duplicates="drop")
    agg = dfp.groupby("bucket", observed=True).agg(positives=("y","sum"), total=("y","size")).reset_index()
    agg["cum_pos"] = agg["positives"].cumsum()
    agg["cum_total"] = agg["total"].cumsum()
    total_pos = max(int(dfp["y"].sum()), 1)
    agg["perc_samples"] = agg["cum_total"] / len(dfp)
    agg["perc_pos"] = agg["cum_pos"] / total_pos
    agg["lift"] = (agg["perc_pos"] / agg["perc_samples"]).replace([np.inf, np.nan], 0.0)
    return agg

rel = reliability_curve(y_te, proba_best, bins=10)
fig = go.Figure()
fig.add_scatter(x=rel["mean_pred"], y=rel["frac_pos"], mode="lines+markers", name="Reliability")
fig.add_scatter(x=[0,1], y=[0,1], mode="lines", name="Perfect", line=dict(dash="dash", color="#888"))
fig.update_layout(title=f"Reliability Curve — {best_name} (calibrated)", xaxis_title="Mean predicted prob", yaxis_title="Fraction positives")
fig.show()

lift = lift_gain_curves(y_te, proba_best, bins=20)
fig = go.Figure()
fig.add_scatter(x=lift["perc_samples"], y=lift["lift"], mode="lines+markers", name="Lift")
fig.update_layout(title=f"Lift Curve — {best_name} (calibrated)", xaxis_title="% of samples (cumulative)", yaxis_title="Lift")
fig.show()

# ---- Persist basic artifacts ----
pd.DataFrame({"y_true": y_te.values, "proba_best": proba_best}).to_csv("artifacts/metrics/test_predictions.csv", index=False)
with open("artifacts/metrics/model_manifest.json","w") as f:
    json.dump({"best_model": best_name, "auc_cv": float(SCORES[best_name]["auc_cv"]),
               "auc_test_cal": float(cal_auc), "ap_test_cal": float(cal_ap)}, f, indent=2)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:43:55] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:43:55] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:43:55] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:43:55] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [17:44:01] WARNING: /workspace/src/context.cc:44: No visible GPU is found, setting device to CPU.


In [7]:
# ===================== B) Decision Thresholding — Cost Minimization + Capacity Constraint =====================
import numpy as np, pandas as pd, json, plotly.graph_objects as go, os

section("Decision Thresholding", "Minimize total cost • Explore trade-offs • Optional capacity constraint")

# ---- Guards ----
assert all(k in globals() for k in ["y_te","proba_best","best_name","CFG","show_list"]), \
    "Run the Modeling & Evaluation block first."

COST_FP        = CFG.get("COST_FP", 1.0)
COST_FN        = CFG.get("COST_FN", 8.0)
BENEFIT_TP     = CFG.get("BENEFIT_TP", 0.0)
MAX_REVIEW_RATE= CFG.get("MAX_REVIEW_RATE", None)

# ---- Utilities ----
def metrics_over_thresholds(y_true, proba, grid):
    rows=[]
    y = np.asarray(y_true, int); p = np.asarray(proba, float)
    for t in grid:
        pred = (p >= t).astype(int)
        tp = int(((pred==1)&(y==1)).sum())
        fp = int(((pred==1)&(y==0)).sum())
        fn = int(((pred==0)&(y==1)).sum())
        tn = int(((pred==0)&(y==0)).sum())
        prec = tp / (tp + fp + 1e-9)
        rec  = tp / (tp + fn + 1e-9)
        f1   = 2*prec*rec / (prec+rec + 1e-9)
        cost = COST_FP*fp + COST_FN*fn - BENEFIT_TP*tp
        rows.append((float(t), prec, rec, f1, cost, tp, fp, fn, tn, float(pred.mean())))
    return pd.DataFrame(rows, columns=["t","precision","recall","f1","cost","TP","FP","FN","TN","rate"])

# ---- Compute curves ----
thr_grid = np.linspace(0, 1, 401)
df_thr   = metrics_over_thresholds(y_te, proba_best, thr_grid)
best_f1_row = df_thr.loc[df_thr["f1"].idxmax()]
best_f1_thr = float(best_f1_row["t"])

cost_df = df_thr[["t","TP","FP","FN","TN","cost","rate"]].rename(columns={"t":"threshold","cost":"total_cost","rate":"pred_positive_rate"})
best_cost_row = cost_df.loc[cost_df["total_cost"].idxmin()]
BEST_COST_THR = float(best_cost_row["threshold"])

# ---- Optional capacity constraint ----
CAP_THR = None
if MAX_REVIEW_RATE is not None:
    try:
        max_rate = float(MAX_REVIEW_RATE)
        if 0 <= max_rate <= 1.0:
            feasible = cost_df[cost_df["pred_positive_rate"] <= max_rate]
            if not feasible.empty:
                CAP_THR = float(feasible.loc[feasible["total_cost"].idxmin(), "threshold"])
    except Exception:
        pass

# ---- Summary ----
summary = [
    f"Costs → FP={COST_FP:.2f}, FN={COST_FN:.2f}, TP_benefit={BENEFIT_TP:.2f}",
    f"Min-cost threshold: {BEST_COST_THR:.2f} | Total cost: {best_cost_row['total_cost']:.2f} | Rate: {best_cost_row['pred_positive_rate']:.3f}",
    f"Best-F1 threshold: {best_f1_thr:.2f} | F1={best_f1_row['f1']:.3f} | Precision={best_f1_row['precision']:.3f} | Recall={best_f1_row['recall']:.3f}"
]
if CAP_THR is not None and abs(CAP_THR - BEST_COST_THR) > 1e-12:
    cap_row = cost_df.iloc[(cost_df["threshold"] - CAP_THR).abs().idxmin()]
    summary.append(f"Capacity threshold (≤ {float(MAX_REVIEW_RATE):.2f} rate): {CAP_THR:.2f} | Cost: {cap_row['total_cost']:.2f} | Rate: {cap_row['pred_positive_rate']:.3f}")
show_list("Decision Summary", summary)

# ---- Plots ----
fig = go.Figure()
fig.add_scatter(x=cost_df["threshold"], y=cost_df["total_cost"], name="Total Cost", mode="lines")
fig.add_vline(x=BEST_COST_THR, line_dash="dash", annotation_text="Min Cost", annotation_position="top left")
if CAP_THR is not None and abs(CAP_THR - BEST_COST_THR) > 1e-12:
    fig.add_vline(x=CAP_THR, line_dash="dot", annotation_text="Capacity", annotation_position="top right")
fig.update_layout(title="Cost Curve — Total Cost vs Threshold", xaxis_title="Threshold", yaxis_title="Total Cost")
fig.show()

fig = go.Figure()
fig.add_scatter(x=df_thr["t"], y=df_thr["precision"], name="Precision", mode="lines")
fig.add_scatter(x=df_thr["t"], y=df_thr["recall"],    name="Recall",    mode="lines")
fig.add_scatter(x=df_thr["t"], y=df_thr["f1"],        name="F1",        mode="lines")
fig.add_scatter(x=df_thr["t"], y=df_thr["cost"],      name="Total Cost", mode="lines", yaxis="y2")
fig.update_layout(
    title=f"Threshold Explorer — {best_name} (calibrated)",
    xaxis_title="Threshold",
    yaxis_title="F1 / Precision / Recall",
    yaxis2=dict(title="Total Cost", overlaying="y", side="right")
)
fig.show()

# ---- Save artifacts ----
os.makedirs("artifacts/decision", exist_ok=True)
cost_df.to_csv("artifacts/decision/decision_cost_curve.csv", index=False)
df_thr.to_csv("artifacts/decision/threshold_explorer.csv", index=False)
pd.DataFrame([{
    "best_cost_threshold": BEST_COST_THR,
    "capacity_threshold": None if CAP_THR is None else CAP_THR,
    "best_f1_threshold": best_f1_thr
}]).to_csv("artifacts/decision/thresholds_summary.csv", index=False)

with open("artifacts/decision/decision_manifest.json", "w") as f:
    json.dump({
        "best_model": best_name,
        "costs": {"FP": float(COST_FP), "FN": float(COST_FN), "TP_benefit": float(BENEFIT_TP)},
        "best_cost_threshold": float(BEST_COST_THR),
        "capacity_threshold": None if CAP_THR is None else float(CAP_THR),
        "max_review_rate": None if MAX_REVIEW_RATE is None else float(MAX_REVIEW_RATE),
        "best_f1_threshold": float(best_f1_thr)
    }, f, indent=2)


In [8]:
# ===================== Permutation Importance (AUC-Based • Self-Healing) =====================
section(
    "Permutation Importance",
    "🔁 Global feature ranking via shuffling • ✅ AUC-based • 🛡️ Robust fallback logic"
)

import os, json, time, math, warnings, platform
import numpy as np, pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import HTML, display, Image
from sklearn.metrics import roc_auc_score

# 📌 Ensure required sklearn tool
try:
    from sklearn.inspection import permutation_importance
except Exception as e:
    raise ImportError("❌ Missing `sklearn.inspection.permutation_importance` — upgrade scikit-learn.") from e

# ---- Kaggle-safe rendering ----
pio.renderers.default = "notebook_connected"

def kaggle_show(fig):
    # 1) Native renderer
    try:
        fig.show()
        return
    except Exception:
        pass
    # 2) Static PNG fallback 
    try:
        png = fig.to_image(format="png", width=900, height=600, scale=2)
        display(Image(png))
        return
    except Exception:
        pass
    # 3) Last resort: inline HTML (may be sanitized by Kaggle)
    html = pio.to_html(fig, include_plotlyjs=True, full_html=False)
    display(HTML(html))

t0 = time.time()
warnings.filterwarnings("ignore")

# ------------ Config & globals heal ------------
SEED = int(globals().get("SEED", 42))
N_JOBS = int(globals().get("N_JOBS", -1))
CFG = globals().get("CFG", {})
PI_REPEATS = int(CFG.get("PI_REPEATS", 10))
PI_MAX_SAMPLES = CFG.get("PI_MAX_SAMPLES", None)  # e.g., 20000 or None

# Windows often dislikes joblib parallelism with PI; force single core there
if platform.system().lower().startswith("win") and N_JOBS != 1:
    N_JOBS = 1

# UI shims
if "ui" not in globals():
    class _UI:
        def card_text(self, title, body): print(f"[{title}] {body}")
    ui = _UI()

if "show_table" not in globals():
    def show_table(title, df, head=None):
        print(title)
        display(df.head(head) if head else df)

# ------------ Build/validate X_te, y_te ------------
if "X_te" not in globals() or "y_te" not in globals():
    if "df" in globals() and "TARGET" in globals() and TARGET in df.columns:
        from sklearn.model_selection import train_test_split
        y_full = df[TARGET]
        X_full = df.drop(columns=[TARGET])
        X_tr, X_te, y_tr, y_te = train_test_split(
            X_full, y_full, test_size=0.2,
            stratify=y_full if y_full.nunique() <= 20 else None,
            random_state=SEED
        )
        print("✅ Built X_te/y_te via fallback train/test split.")
    else:
        raise RuntimeError("❌ Cannot build X_te/y_te. Define them or ensure df/TARGET exists.")

# force binary 0/1 if labels are {0,1} or {False,True}
y_te_bin = pd.Series(y_te).astype(int).values

# Subsample for speed/memory if requested
if PI_MAX_SAMPLES is not None and len(X_te) > PI_MAX_SAMPLES:
    rng = np.random.default_rng(SEED)
    idx = rng.choice(len(X_te), size=int(PI_MAX_SAMPLES), replace=False)
    X_pi = X_te.iloc[idx].copy()
    y_pi = y_te_bin[idx].copy()
    sampled_note = f"sampled={len(X_pi):,}/{len(X_te):,}"
else:
    X_pi = X_te
    y_pi = y_te_bin
    sampled_note = f"full={len(X_pi):,}"

# ------------ Ensure best_model exists ------------
if "best_model" not in globals():
    raise RuntimeError("❌ `best_model` is undefined. Train or load a fitted estimator/pipeline.")

# ------------ Feature names extraction ------------
def _unwrap_pipeline(est):
    return getattr(est, "base_estimator_", None) or getattr(est, "base_estimator", None) or est

def _get_feature_names(pre, num, cat):
    try:
        num_names = list(np.array(num, dtype=object))
        ohe = pre.named_transformers_["cat"].named_steps.get("ohe", None)
        if ohe is not None and hasattr(ohe, "get_feature_names_out"):
            cat_names = list(ohe.get_feature_names_out(cat))
            return num_names + cat_names
    except Exception:
        pass
    return list(num) + list(cat)

pipe_like = _unwrap_pipeline(best_model)
if ("num_cols" in globals() and "cat_cols" in globals()
    and hasattr(pipe_like, "named_steps") and "pre" in pipe_like.named_steps):
    feat_names = _get_feature_names(pipe_like.named_steps["pre"], num_cols, cat_cols)
else:
    feat_names = list(getattr(X_pi, "columns", [])) or [f"f{i}" for i in range(X_pi.shape[1])]

# ------------ Baseline AUC (for sanity & reporting) ------------
def _predict_scores(est, X):
    if hasattr(est, "predict_proba"):
        return est.predict_proba(X)[:, 1]
    if hasattr(est, "decision_function"):
        s = est.decision_function(X)
        # normalize decision_function to [0,1] for comparability if needed
        try:
            s = (s - s.min()) / (s.max() - s.min() + 1e-12)
        except Exception:
            pass
        return s
    # worst-case: hard labels
    return est.predict(X)

baseline_scores = _predict_scores(best_model, X_pi)
baseline_auc = float(roc_auc_score(y_pi, baseline_scores))

# ------------ Run Permutation Importance ------------
def _auc_manual(est, X, y):
    return roc_auc_score(y, _predict_scores(est, X))

try:
    r = permutation_importance(
        best_model, X_pi, y_pi,
        scoring="roc_auc", n_repeats=PI_REPEATS, n_jobs=N_JOBS, random_state=SEED
    )
except Exception:
    # Fallback to custom scorer (handles decision_function-only models and joblib quirks)
    r = permutation_importance(
        best_model, X_pi, y_pi,
        scoring=_auc_manual, n_repeats=PI_REPEATS, n_jobs=1, random_state=SEED
    )

# ------------ Build importance DataFrame ------------
k = min(len(feat_names), len(r.importances_mean))
pi_df = (pd.DataFrame({
            "feature": feat_names[:k],
            "importance": r.importances_mean[:k],
            "std": r.importances_std[:k] if hasattr(r, "importances_std") else np.nan
        })
        .sort_values("importance", ascending=False)
        .reset_index(drop=True))

# add handy columns
if len(pi_df):
    top_val = max(pi_df["importance"].max(), 1e-12)
    pi_df["rank"] = np.arange(1, len(pi_df) + 1)
    pi_df["pct_of_top"] = (pi_df["importance"] / top_val).clip(0, None)

# ------------ Save artifacts ------------
os.makedirs("artifacts/metrics", exist_ok=True)
pi_path_csv  = "artifacts/metrics/permutation_importance_auc.csv"
pi_path_json = "artifacts/metrics/permutation_importance_top20.json"
meta_path    = "artifacts/metrics/permutation_importance_meta.json"

pi_df.to_csv(pi_path_csv, index=False)
with open(pi_path_json, "w") as f:
    json.dump(pi_df.head(20).to_dict(orient="records"), f, indent=2)

with open(meta_path, "w") as f:
    json.dump({
        "baseline_auc": baseline_auc,
        "repeats": PI_REPEATS,
        "n_jobs": N_JOBS,
        "seed": SEED,
        "rows_used": int(len(X_pi)),
        "rows_total": int(len(X_te)),
        "sampling": sampled_note
    }, f, indent=2)

# ------------ Plot (Kaggle-safe) ------------
if pi_df.empty:
    ui.card_text("Permutation Importance", "No importances computed. Check model and X_pi.")
else:
    plot_top = int(min(40, len(pi_df)))
    fig = px.bar(
        pi_df.head(plot_top).sort_values("importance"),
        x="importance", y="feature", orientation="h",
        title=f"Permutation Importance (AUC Δ) — Top {plot_top} • baseline AUC={baseline_auc:.3f}"
    )
    fig.update_layout(height=600, margin=dict(l=60, r=40, t=60, b=40))
    kaggle_show(fig)

# ------------ Summary cards ------------
ui.card_text(
    "Permutation Importance",
    f"✓ Completed in {time.time()-t0:.2f}s • baseline AUC={baseline_auc:.3f} • repeats={PI_REPEATS} • {sampled_note}"
)
try:
    lb = pd.read_csv("artifacts/metrics/leaderboard_metrics.csv")
    show_table("Top-3 Models by AUC (CV/Test)", lb.head(3), head=3)
except Exception as e:
    ui.card_text("Leaderboard", f"Unavailable — {e}")


model,AUC_CV,AUC_Test,AP_Test,Brier_Test,Gini_Test,Train_s
XGBoost,0.761246,0.778144,0.620240,0.115803,0.546556,13.63
RandomForest,0.759942,0.780338,0.646451,0.106632,0.560677,24.77
LogisticRegression,0.735077,0.776310,0.585926,0.177713,0.552621,5.78


In [9]:
# ===================== Model Export (Lean, Non-duplicative) =====================
section("Save Artifacts", "Lightweight export of model + schema + metadata")

import os, sys, json, time, hashlib, joblib, pandas as pd

# Require previous modeling blocks
assert all(k in globals() for k in ["best_model","X_te","y_te","FEATURES","num_cols","cat_cols","show_list"]), \
    "Run modeling and split blocks first."

os.makedirs("artifacts/models", exist_ok=True)

# Reuse existing calibrated model if already saved by earlier blocks
default_model_path = "artifacts/decision/best_model_calibrated.joblib"
model_path = default_model_path if os.path.exists(default_model_path) else "artifacts/models/best_calibrated.joblib"

# Save only if not already present 
if not os.path.exists(model_path):
    joblib.dump(best_model, model_path, compress=("xz", 3))

def _model_name(m):
    base = getattr(m, "base_estimator_", None) or getattr(m, "base_estimator", None) or m
    if hasattr(base, "named_steps") and "clf" in base.named_steps:
        return base.named_steps["clf"].__class__.__name__
    return base.__class__.__name__

def _schema_meta(df, cols):
    dtypes = {c: str(df[c].dtype) for c in cols if c in df.columns}
    return {"columns": cols, "dtypes": dtypes,
            "hash_md5": hashlib.md5("|".join(cols).encode()).hexdigest()}

def _ohe_categories(m):
    try:
        base = getattr(m, "base_estimator", None) or getattr(m, "base_estimator_", None) or m
        pre = getattr(base, "named_steps", {}).get("pre", None)
        if pre is None: return None
        for name, transformer, cols in getattr(pre, "transformers_", []):
            if name == "cat":
                ohe = transformer.named_steps.get("ohe")
                if hasattr(ohe, "categories_"):
                    return {str(c): [None if pd.isna(x) else str(x) for x in arr]
                            for c, arr in zip(cols, ohe.categories_)}
    except Exception:
        return None

meta = {
    "best_model_name": _model_name(best_model),
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "python": sys.version.split()[0],
    "versions": {"pandas": pd.__version__}
}
try:
    import sklearn, xgboost as xgb
    meta["versions"].update({"scikit_learn": sklearn.__version__, "xgboost": xgb.__version__})
except Exception:
    import sklearn
    meta["versions"].update({"scikit_learn": sklearn.__version__, "xgboost": "N/A"})

meta["target"] = {"name": getattr(y_te, "name", "target"),
                  "test_prevalence": float(pd.Series(y_te).mean())}
meta["feature_count"] = len(FEATURES)
meta["numeric_features_count"] = len(num_cols)
meta["categorical_features_count"] = len(cat_cols)

json.dump(meta, open("artifacts/models/model_info.json","w"), indent=2)
json.dump(_schema_meta(X_te, FEATURES), open("artifacts/models/feature_schema.json","w"), indent=2)
ohe_map = _ohe_categories(best_model)
if ohe_map:
    json.dump(ohe_map, open("artifacts/models/ohe_categories.json","w"), indent=2)

show_list("Saved Artifacts", [
    f"Model → {model_path}",
    "Metadata → artifacts/models/model_info.json",
    "Schema → artifacts/models/feature_schema.json",
    ("OHE Categories → artifacts/models/ohe_categories.json" if ohe_map else "OHE Categories: N/A")
])


In [10]:
# ===================== Inference Smoke Test (Lean & Robust) =====================
section("Inference Smoke Test", "Verify loaded model + schema + sample prediction")

import os, json, joblib, numpy as np, pandas as pd

# Fallbacks for UI
try:
    show_list
except NameError:
    def show_list(title, items):
        print(f"[{title}]"); [print("-", i) for i in items]
try:
    ui
except NameError:
    class _UI:
        def card_text(self, t, b): print(f"[{t}] {b}")
    ui = _UI()

# Resolve model path (try both decision/ and models/)
CANDIDATE_MODELS = [
    "artifacts/decision/best_model_calibrated.joblib",
    "artifacts/models/best_calibrated.pkl",
    "artifacts/models/best_calibrated.joblib",
]
model_path = next((p for p in CANDIDATE_MODELS if os.path.exists(p)), None)
assert model_path is not None, "No saved model found. Run modeling/export blocks first."

# Load model & schema (fallback to X_te columns if schema is absent)
model = joblib.load(model_path)
schema_path = "artifacts/models/feature_schema.json"
cols = json.load(open(schema_path))["columns"] if os.path.exists(schema_path) else list(X_te.columns)

# Validate presence and order
missing = [c for c in cols if c not in X_te.columns]
extra   = [c for c in X_te.columns if c not in cols]
if missing:
    raise ValueError(f"Missing columns in X_te (first 10): {missing[:10]}")

X_sample = X_te.loc[:, [c for c in cols if c in X_te.columns]].iloc[:5].copy()

def _predict_scores(est, X):
    if hasattr(est, "predict_proba"):
        return est.predict_proba(X)[:, 1]
    if hasattr(est, "decision_function"):
        s = est.decision_function(X)
        s = (s - np.min(s)) / (np.ptp(s) + 1e-12)
        return s
    return est.predict(X)

proba = _predict_scores(model, X_sample)
assert np.isfinite(proba).all(), "Non-finite values from inference."

# Save a compact report
os.makedirs("artifacts/inference", exist_ok=True)
rep = {
    "model_path": model_path,
    "n_features_expected": len(cols),
    "n_features_in_X_te": X_te.shape[1],
    "n_missing": len(missing),
    "n_extra": len(extra),
    "sample_size": len(X_sample),
    "proba_min": float(np.min(proba)),
    "proba_max": float(np.max(proba)),
    "proba_mean": float(np.mean(proba)),
}
json.dump(rep, open("artifacts/inference/smoke_test_report.json", "w"), indent=2)

show_list("Inference Smoke Test", [
    f"Model: {model_path}",
    f"Features (expected/in X_te): {len(cols)}/{X_te.shape[1]}",
    f"Missing: {len(missing)} | Extra: {len(extra)}",
    f"Sample size: {len(X_sample)}",
    f"Proba stats: min={rep['proba_min']:.3f}, mean={rep['proba_mean']:.3f}, max={rep['proba_max']:.3f}",
])
ui.card_text("Inference", "Artifacts loaded and inference successful ✅")


In [11]:
from IPython.display import HTML, display
import os, math
import pandas as pd
import numpy as np

# ---- Minimal guards ----
required = ["best_name","cal_auc","cal_ap","BEST_COST_THR","cost_df"]
missing = [g for g in required if g not in globals()]
assert not missing, f"Run previous blocks first. Missing: {missing}"

# ---- Heal FEATURES / num_cols / cat_cols if missing ----
if "FEATURES" not in globals():
    if "X_te" in globals():
        FEATURES = list(X_te.columns)
    elif "X_tr" in globals():
        FEATURES = list(X_tr.columns)
    else:
        FEATURES = []

if "num_cols" not in globals() or "cat_cols" not in globals():
    _num, _cat = [], []
    if "X_te" in globals() and len(FEATURES):
        _num = X_te[FEATURES].select_dtypes(include="number").columns.tolist()
        _cat = [c for c in FEATURES if c not in _num]
    elif "df" in globals() and len(FEATURES):
        _num = df[FEATURES].select_dtypes(include="number").columns.tolist()
        _cat = [c for c in FEATURES if c not in _num]
    num_cols = globals().get("num_cols", _num)
    cat_cols = globals().get("cat_cols", _cat)

# ---- Ensure best_thr exists (df_thr -> thresholds_report -> fallback) ----
if "best_thr" not in globals():
    _bt = None
    try:
        if "df_thr" in globals() and {"t","f1"}.issubset(set(df_thr.columns)):
            _bt = float(df_thr.loc[df_thr["f1"].idxmax(), "t"])
    except Exception:
        pass
    if _bt is None:
        try:
            if all(k in globals() for k in ["y_te","proba_best","thresholds_report"]):
                _tmp = thresholds_report(y_te, proba_best, steps=401)
                _bt = float(_tmp.loc[_tmp["f1"].idxmax(), "threshold"])
        except Exception:
            pass
    if _bt is None:
        _bt = float(BEST_COST_THR)
    best_thr = _bt

# ---- Normalize/Heal cost_df columns (idempotent) ----
cost_df = cost_df.copy()

# pred_positive_rate -> compute if missing from counts
if "pred_positive_rate" not in cost_df.columns and {"TP","FP","FN","TN"}.issubset(cost_df.columns):
    n_series = (cost_df[["TP","FP","FN","TN"]].sum(axis=1)).replace(0, 1)
    cost_df["pred_positive_rate"] = (cost_df["TP"] + cost_df["FP"]) / n_series

# cost_per_obs / net_cost_per_1k -> compute if missing
need_cost_cols = ("net_cost_per_1k" not in cost_df.columns) or ("cost_per_obs" not in cost_df.columns)
if need_cost_cols:
    if {"TP","FP","FN","TN"}.issubset(cost_df.columns):
        n_series = (cost_df[["TP","FP","FN","TN"]].sum(axis=1)).replace(0, 1)
    elif "y_te" in globals():
        n_series = pd.Series([max(len(y_te),1)]*len(cost_df), index=cost_df.index)
    else:
        n_series = pd.Series([1]*len(cost_df), index=cost_df.index)
    if "cost_per_obs" not in cost_df.columns:
        cost_df["cost_per_obs"] = cost_df["total_cost"] / n_series
    if "net_cost_per_1k" not in cost_df.columns:
        cost_df["net_cost_per_1k"] = 1000.0 * cost_df["cost_per_obs"]

# ---- Extract best row around BEST_COST_THR ----
best_thr_val = float(BEST_COST_THR)
_best_row = cost_df.iloc[(cost_df["threshold"] - best_thr_val).abs().idxmin()]

def _fmt(x, nd=3):
    try: return f"{float(x):.{nd}f}"
    except Exception: return str(x)

# Safe pulls with defaults
tp = int(_best_row.get("TP", 0))
fp = int(_best_row.get("FP", 0))
fn = int(_best_row.get("FN", 0))
tn = int(_best_row.get("TN", 0))
n_row = max(tp + fp + fn + tn, 1)

_total_cost = float(_best_row.get("total_cost", 0.0))
_cost      = _fmt(_total_cost, 2)
_cost_1k   = _fmt(float(_best_row.get("net_cost_per_1k", 1000.0 * (_total_cost / n_row))), 2)
rate_pp    = _fmt(_best_row.get("pred_positive_rate", (tp + fp) / n_row), 3)

# Optional capacity threshold line
cap_line = ""
if "CAP_THR" in globals() and CAP_THR is not None and abs(float(CAP_THR) - best_thr_val) > 1e-12:
    _cap_row = cost_df.iloc[(cost_df["threshold"] - float(CAP_THR)).abs().idxmin()]
    cap_line = (
        f"<li>Capacity-aware threshold: <b>{_fmt(CAP_THR,2)}</b> "
        f"(rate={_fmt(_cap_row.get('pred_positive_rate', np.nan),3)}, "
        f"total cost={_fmt(_cap_row.get('total_cost', np.nan),2)})</li>"
    )

# ---- Artifacts list (soft checks) ----
artifacts = []
for p in [
    "artifacts/metrics/leaderboard_metrics.csv",
    "artifacts/metrics/permutation_importance_auc.csv",
    "artifacts/decision/decision_cost_curve.csv",
    "artifacts/decision/decision_manifest.json",
    "artifacts/models/model_info.json",
]:
    if os.path.exists(p): artifacts.append(p.replace("artifacts/",""))

art_list_html = "".join([f'<li><code>artifacts/{p}</code></li>' for p in artifacts]) or "<li>No artifacts found.</li>"

# ---- HTML summary (TVS style) ----
html = f"""
<div style="
  --ink:#eaf2ff; --muted:#a6b9d3; --panel:rgba(255,255,255,.06); --border:rgba(255,255,255,.16);
  --accent:#60a5fa; --accent2:#22d3ee; --glow:rgba(96,165,250,.40);
  background:
    radial-gradient(1200px 600px at -10% -20%, #0ea5e9 0%, #0ea5e920 40%, #0000 70%),
    radial-gradient(900px 500px at 110% 10%, #2563eb 0%, #2563eb25 45%, #0000 70%),
    linear-gradient(135deg, #0b1021 0%, #0b1428 55%, #0b1021 100%);
  color:var(--ink); border-radius:20px; padding:28px 30px; position:relative; overflow:hidden;
  box-shadow:0 18px 46px rgba(0,0,0,.45), inset 0 0 80px rgba(255,255,255,.04);
  border:1px solid var(--border); backdrop-filter:blur(14px); margin:22px 0;">

  <div style="position:absolute; inset:-2px; pointer-events:none;
              background: radial-gradient(1200px 1200px at 0% 100%, var(--glow), #0000 55%); opacity:.22;"></div>

  <h2 style="margin:0 0 6px 0; font-size:22px; letter-spacing:.2px;">
    🌌 <span style="color:#dbeafe;">Final Summary — Digital Wellbeing Risk Intelligence</span>
  </h2>

  <p style="font-size:15.5px; line-height:1.75; opacity:.96; margin:8px 0 10px; max-width:960px;">
    This notebook turns behavioral patterns into <b>actionable wellbeing intelligence</b> through a reproducible, production-ready ML workflow.
  </p>

  <hr style="border:none;height:1px;background:linear-gradient(to right,#0000,rgba(96,165,250,.35),#0000);margin:14px 0;">

  <ul style="font-size:14.5px; line-height:1.8; opacity:.94; padding-left:18px; margin:0;">
    <li>Model: <b>{best_name}</b> • Calibrated (isotonic)</li>
    <li>Test AUC: <b>{cal_auc:.4f}</b> • Average Precision: <b>{cal_ap:.4f}</b> • Best F1 threshold: <b>{best_thr:.2f}</b></li>
    <li>Cost-minimizing threshold: <b>{best_thr_val:.2f}</b> → TP={tp}, FP={fp}, FN={fn}, TN={tn}, rate={rate_pp}, total cost={_cost}, net/1k={_cost_1k}</li>
    {cap_line}
    <li>Features: <b>{len(FEATURES)}</b> total — {len(num_cols)} numeric, {len(cat_cols)} categorical</li>
  </ul>

  <p style="font-size:15px; line-height:1.7; opacity:.95; margin:14px 0 0; max-width:960px;">
    Artifacts for traceability and deployment:
  </p>
  <ul style="font-size:14px; line-height:1.8; opacity:.94; padding-left:18px; margin:6px 0 0;">
    {art_list_html}
  </ul>

  <div style="display:flex; flex-wrap:wrap; gap:8px; margin:16px 0 0;">
    <span style="font-size:11.5px; padding:6px 10px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Leak-Safe</span>
    <span style="font-size:11.5px; padding:6px 10px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Plotly EDA</span>
    <span style="font-size:11.5px; padding:6px 10px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Calibrated</span>
    <span style="font-size:11.5px; padding:6px 10px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Cost-Aware</span>
    <span style="font-size:11.5px; padding:6px 10px; border-radius:999px;
      background:rgba(255,255,255,.08); border:1px solid var(--border);">Reproducible</span>
  </div>
</div>
"""
display(HTML(html))
